In [1]:
pip install transformers pandas numpy torch scikit-learn

In [4]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW  # Changed import location
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm


In [6]:
# Load the dataset
df = pd.read_csv(r'C:\Users\naman\chat_dataset.csv')
print(df.head())
print(f"Dataset size: {len(df)}")
print(df['sentiment'].value_counts())

In [ ]:
# Map sentiments to numerical values
sentiment_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}
df['label'] = df['sentiment'].map(sentiment_mapping)

# Split data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['message'], df['label'],
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the texts
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

In [12]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=3  # negative, neutral, positive
)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

In [13]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training function
def train(model, train_loader, optimizer, device, epochs=3):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}")

# Train the model
train(model, train_loader, optimizer, device)

In [ ]:
def evaluate(model, test_loader, device):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    print(classification_report(true_labels, predictions, target_names=sentiment_mapping.keys()))

    return predictions

# Evaluate the model
predictions = evaluate(model, test_loader, device)

In [ ]:
def predict_sentiment(text, model, tokenizer, device):
    # Preprocess the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Make prediction
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    # Get predicted class
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    # Map back to sentiment label
    inverse_mapping = {v: k for k, v in sentiment_mapping.items()}
    sentiment = inverse_mapping[predicted_class]

    # Get confidence scores
    probabilities = torch.softmax(logits, dim=1)[0]
    confidence = probabilities[predicted_class].item()

    return sentiment, confidence

In [ ]:
# Example usage
text = "I really enjoyed the movie, it was fantastic!"
sentiment, confidence = predict_sentiment(text, model, tokenizer, device)
print(f"Text: {text}")
print(f"Predicted sentiment: {sentiment} (confidence: {confidence:.2f})")